In [ ]:
import os
import numpy as np
import pandas as pd
import tempfile
import imageio.v3 as iio
from skimage.io import imread
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from skimage.feature import local_binary_pattern
from radiomics import featureextractor

In [ ]:
# ---- Directories ----
IMG_DIR = "images"
MASK_DIR = "predicted_masks"


In [ ]:
# ---- Load PyRadiomics extractor ----
extractor = featureextractor.RadiomicsFeatureExtractor("radiomics_params.yaml")


In [ ]:
radiomics_list = []
lbp_list = []
combined_list = []
ids = []

In [ ]:
# ---- LBP settings ----
LBP_RADIUS = 2
LBP_POINTS = 8 * LBP_RADIUS
LBP_METHOD = "uniform"

In [ ]:
image_files = sorted(os.listdir(IMG_DIR))

In [ ]:
for filename in image_files:
    patient_id = os.path.splitext(filename)[0]
    img_path = os.path.join(IMG_DIR, filename)

    # ---- Find mask ----
    mask_candidates = [
        f"{patient_id}_mask.png",
        f"{patient_id}_pred_mask.png"
    ]
    mask_path = None
    for m in mask_candidates:
        p = os.path.join(MASK_DIR, m)
        if os.path.exists(p):
            mask_path = p
            break
    if mask_path is None:
        print(f"No mask found for {patient_id}, skipping.")
        continue

    # ---- Load mask (assume 0/1 or 0/255) ----
    mask_bin = imread(mask_path)
    if mask_bin.ndim == 3:
        mask_bin = mask_bin[:, :, 0]
    mask_bin = np.where(mask_bin > 0, 1, 0)  # ensure 0/1

    # ---- Temporary mask for PyRadiomics ----
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp_mask:
        tmp_mask_path = tmp_mask.name
    iio.imwrite(tmp_mask_path, mask_bin.astype(np.uint8))

    # ---- Extract PyRadiomics features ----
    radiomics_raw = extractor.execute(img_path, tmp_mask_path, label=1)
    os.remove(tmp_mask_path)
    radiomics_features = {k: v for k, v in radiomics_raw.items() if not k.startswith("diagnostics")}

    # ---- Extract LBP features ----
    img = imread(img_path)
    if img.ndim == 3:
        img = rgb2gray(img)
    img = img_as_ubyte(img)

    lbp = local_binary_pattern(img, LBP_POINTS, LBP_RADIUS, LBP_METHOD)
    lbp_region = lbp[mask_bin.astype(bool)]
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp_region, bins=n_bins, range=(0, n_bins), density=True)

    lbp_features = {f"LBP_hist_bin_{i}": hist[i] for i in range(len(hist))}
    lbp_features["LBP_mean"] = lbp_region.mean()
    lbp_features["LBP_std"] = lbp_region.std()
    lbp_features["LBP_entropy"] = -np.sum(hist * np.log2(hist + 1e-10))

    # ---- Save features ----
    radiomics_list.append({"ID": patient_id, **radiomics_features})
    lbp_list.append({"ID": patient_id, **lbp_features})
    combined_list.append({"ID": patient_id, **radiomics_features, **lbp_features})

# ---- Convert to DataFrames ----
df_radiomics = pd.DataFrame(radiomics_list)
df_lbp = pd.DataFrame(lbp_list)
df_combined = pd.DataFrame(combined_list)

# ---- Save CSVs ----
df_radiomics.to_csv("radiomics_only.csv", index=False)
df_lbp.to_csv("lbp_only.csv", index=False)
df_combined.to_csv("radiomics_with_lbp.csv", index=False)

print("Saved: radiomics_only.csv, lbp_only.csv, radiomics_with_lbp.csv")